### When running this notebook via the Galaxy portal
You can access your data via the dataset number. Using a Python kernel, you can access dataset number 42 with ``handle = open(get(42), 'r')``.
To save data, write your data to a file, and then call ``put('filename.txt')``. The dataset will then be available in your galaxy history.
<br><br>Note that if you are putting/getting to/from a different history than your default history, you must also provide the history-id.
<br><br>More information including available galaxy-related environment variables can be found at https://github.com/bgruening/docker-jupyter-notebook. This notebook is running in a docker container based on the Docker Jupyter container described in that link.


# Data Handling (Python) 


In [1]:
import ROOT as R
import import_ipynb
import setPath
from os import listdir
from os.path import isfile, join
from Input.OpenDataPandaFramework13TeV import *
%jsroot on

Welcome to JupyROOT 6.24/02
importing Jupyter notebook from setPath.ipynb
importing Jupyter notebook from /storage/galaxy/jobs_directory/003/3098/working/jupyter/Input/OpenDataPandaFramework13TeV.ipynb
This library contains handy functions to ease the access and use of the 13TeV ATLAS OpenData release

getBkgCategories()
	 Dumps the name of the various background cataegories available 
	 as well as the number of samples contained in each category.
	 Returns a vector with the name of the categories

getSamplesInCategory(cat)
	 Dumps the name of the samples contained in a given category (cat)
	 Returns dictionary with keys being DSIDs and values physics process name from filename.

getMCCategory()
	 Returns dictionary with keys DSID and values MC category

initialize(indir)
	 Collects all the root files available in a certain directory (indir)



Setting luminosity to 10064 pb^-1

###############################
#### Background categories ####
###############################
Category    

## 1. Reading the dataset

Set the analaysis to run (*1largeRjet1lep*, *1lep1tau*, *3lep*, *exactly2lep*, *GamGam*, *2lep*, *4lep*)

Set the directory where you have downloaded the ATLAS OpenData samples you want to run over

In [2]:
opendatadir = "/storage/shared/data/fys5555/ATLAS_opendata/"
analysis = "2lep"

In [3]:
background = R.TChain("mini")
data = R.TChain("mini")

A list of all the background samples, category and their IDs can be found in **Infofile.txt**. The cross-section, efficiencies etc. needed for scaling are stored in the **Files_<---->**. We read these files and add all the samples to the TChain. We also (for later convenience) make a vector containing the dataset IDs. 

In [4]:
mcfiles = initialize(opendatadir+"/"+analysis+"/MC")
datafiles = initialize(opendatadir+"/"+analysis+"/Data")
allfiles = z = {**mcfiles, **datafiles}
Backgrounds = getBkgCategories() 

####################################################################################################
BACKGROUND SAMPLES
####################################################################################################
WARNING 	 File for ggH125_tautaulh not found in /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC
WARNING 	 File for VBFH125_tautaulh not found in /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC
####################################################################################################
SIGNAL SAMPLES
####################################################################################################
WARNING 	 File for ttH125_gamgam not found in /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC
WARNING 	 File for ggH125_gamgam not found in /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC
WARNING 	 File for VBFH125_gamgam not found in /storage/shared/data/fys5555/ATLAS_opendata//2lep/MC
WARNING 	 File for WpH125J_Wincl_gamgam not found in /storage

In [5]:
MCcat = {}
for cat in allfiles:
    for dsid in allfiles[cat]["dsid"]:
        try:
            MCcat[int(dsid)] = cat
        except:
            continue

In [6]:
dataset_IDs = []
background.Reset()
for b in Backgrounds:
    i = 0
    if not b in mcfiles.keys(): continue
    for mc in mcfiles[b]["files"]:
        if not os.path.isfile(mc): continue
        try:
            dataset_IDs.append(int(mcfiles[b]["dsid"][i]))
            background.Add(mc)
        except:
            print("Could not get DSID for %s. Skipping"%mc)
        i += 1
nen = background.GetEntries()
print("Added %i entries for backgrounds"%(nen))

Added 118288518 entries for backgrounds


In [7]:
data.Reset(); 
for d in datafiles["data"]["files"]:  
    if not os.path.isfile(d): continue
    data.Add(d)
nen = data.GetEntries()
print("Added %i entries for data"%(nen))

Added 24411580 entries for data


## 2. Event selection

For machine learning using (semi)-unsupervised learning, we need as unbiased data as possible. Thus, we need to construct a dataframe with all the neccesary features, which is a lot of data.

In [8]:
# Retrieve lumi from library
%store -r lumi

l1 = R.TLorentzVector()
l2 = R.TLorentzVector()

dilepton = R.TLorentzVector()


no stored variable or alias lumi


This is the cell where the analysis is performed. Note that the cell needs to be run twice:

1. with data = 0 to run over MC
2. with data = 1 to run over data

Note that the MC running takes ~5 minutes for 3lep analysis. Much(!!!) more time for e.g. 2lep analysis! Data running is relatively fast for 3lep. 

In [9]:
import pandas as pd
import numpy as np

In [ ]:
%%time
import time


isData = 0; 

if isData == 1: ds = data 
else: ds = background

legal_flavor_tot = [13*2, 11*2]

columns = {"met":[], "XSection":[], 
           "tot_lep_invariant_mass":[], "lep0_pt":[], "lep1_pt":[], 
           "lep_z0_0":[], "lep_z0_1":[], "lep0_E":[], "lep1_E":[],
           "lep_ptcone30_0":[],"lep_ptcone30_1":[], 
           "lep_etcone20_0":[], "lep_etcone20_1":[],
           "lep_eta0":[], "lep_phi0":[], "lep_eta1":[], "lep_phi1":[],
           "lep_trackd0pvunbiased_0":[], "lep_trackd0pvunbiased_1":[],
           "lep_tracksigd0pvunbiased_0":[], "lep_tracksigd0pvunbiased_1":[],
           "lep_type":[], "jet_n":[], "largeRjet_n":[],"tau_n":[] , "mcWeight":[]
          }

lep0 = 0
lep1 = 1

i = 0   
for event in ds:
    if i == 100:
        break
    if i%10000 == 0 and i>0: 
        print("Total events {:.3f}%".format(i/ds.GetEntries()*100))

    i += 1 
    # First event selection, require charge conservation, and lep flavor, and 
    # to only have the two largest leptons    
    

    ## Cut #1: Require (exactly) 2 leptons
    if not ds.lep_n == 2: continue
    ## Cut #2: Require opposite charge
    if not ds.lep_charge[0] + ds.lep_charge[1] == 0 : continue
    ## Cut #3: Require same flavour (2 electrons or 2 muons)
    if not ds.lep_type[0] + ds.lep_type[1] in legal_flavor_tot: continue

    
    ## Require "good leptons": 
    
    if ds.lep_pt[0]/1000.0 < 25: continue
    if ds.lep_etcone20[0]/ds.lep_pt[0] > 0.30: continue
    if ds.lep_ptcone30[0]/ds.lep_pt[0] > 0.30: continue
    #if not (ds.lep_flag[0] & 512): continue
        
    if ds.lep_pt[1]/1000.0 < 25: continue
    if ds.lep_etcone20[1]/ds.lep_pt[1] > 0.30: continue
    if ds.lep_ptcone30[1]/ds.lep_pt[1] > 0.30: continue

    
    l1.SetPtEtaPhiE(ds.lep_pt[0]/1000., ds.lep_eta[0],
                    ds.lep_phi[0], ds.lep_E[0]/1000.)
    l2.SetPtEtaPhiE(ds.lep_pt[1]/1000., ds.lep_eta[1],
                    ds.lep_phi[1], ds.lep_E[1]/1000.)
    
    dilepton = l1 + l2 
    
    dilep_inv_mass = dilepton.M()
    
    
    ### General information 
    ### General information 
    columns["met"].append(ds.met_et/1000.0)
    columns["XSection"].append(ds.XSection)  
    columns["mcWeight"].append(ds.mcWeight)
    
    ### Lep information
    
    columns["tot_lep_invariant_mass"].append(dilep_inv_mass)
    
    columns["lep0_pt"].append((ds.lep_pt[lep0])/1000)
    columns["lep1_pt"].append((ds.lep_pt[lep1])/1000)
    
    columns["lep0_E"].append((ds.lep_E[lep0])/1000)
    columns["lep1_E"].append((ds.lep_E[lep1])/1000)
    
    columns["lep_ptcone30_0"].append((ds.lep_ptcone30[lep0])/(ds.lep_pt[lep0]))
    columns["lep_ptcone30_1"].append((ds.lep_ptcone30[lep1])/(ds.lep_pt[lep1]))
    
    columns["lep_etcone20_0"].append((ds.lep_etcone20[lep0])/(ds.lep_pt[lep0]))
    columns["lep_etcone20_1"].append((ds.lep_etcone20[lep1])/(ds.lep_pt[lep1]))
    
    columns["lep_eta0"].append((ds.lep_eta[lep0]))
    columns["lep_eta1"].append((ds.lep_eta[lep1]))
    
    columns["lep_phi0"].append((ds.lep_phi[lep0]))
    columns["lep_phi1"].append((ds.lep_phi[lep1]))
    
    columns["lep_z0_0"].append(ds.lep_z0[lep0])
    columns["lep_z0_1"].append(ds.lep_z0[lep1])
    
    columns["lep_trackd0pvunbiased_0"].append(ds.lep_trackd0pvunbiased[lep0])
    columns["lep_trackd0pvunbiased_1"].append(ds.lep_trackd0pvunbiased[lep1])
    
    columns["lep_tracksigd0pvunbiased_0"].append(ds.lep_tracksigd0pvunbiased[lep0])
    columns["lep_tracksigd0pvunbiased_1"].append(ds.lep_tracksigd0pvunbiased[lep1])

    columns["lep_type"].append(ds.lep_type[lep0])
    columns["jet_n"].append(ds.jet_n)
    columns["largeRjet_n"].append(ds.largeRjet_n)
    columns["tau_n"].append(ds.tau_n)

print("hello")
df = pd.DataFrame(columns)
    

print("Done!")
if isData == 0:
    df.to_hdf('mcdata.h5', key='df', mode='w')
    print("Remebered to run over data? No? Set data = 1 at the top and run again")
else:
    df.to_hdf('testdata.h5', key='df', mode='w')
    print("Remebered to run over MC? No? Set data = 0 at the top and run again")


In [11]:
df = pd.read_hdf("mcdata.h5")

FileNotFoundError: File mcdata.h5 does not exist

In [ ]:
print(df)